## Step 1: Build dictionaries from Ego4d taxonomy and load data

In [1]:
from preprocessing import build_dictionary
path = 'data/fho_lta_taxonomy.json'
id2verb,id2noun,verb2id,noun2id = build_dictionary(path)

In [2]:
#load data files sampled from Ego4d annotation files
nseg = 8
import json
f = open(f'../v1_data/train_words_nseg{nseg}.json')
train_data = json.load(f)
f = open(f'../v1_data/val_words_nseg{nseg}.json')
val_data = json.load(f)

## Step 2: Build datasets from fine-tuning GPT3

In [ ]:
from build_finetune_ds import build_finetune_dataset
train_ds,val_ds = build_finetune_dataset([train_data,val_data],save_jsonl=False)
#If you set save_jsonl=True, there will be two jsonl file saved on disk.            
#You can now use the following command to submit the files to openai for fine-tuning

###You need to setup openai apikey in environment###
#https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety

####preparing files using openai's tool####
#!openai tools fine_tunes.prepare_data -f train_nseg8.jsonl -q
#!openai tools fine_tunes.prepare_data -f val_nseg8.jsonl -q

####submit to openai for finetune####
#!openai api fine_tunes.create -t "train_nseg8.jsonl" -v "val_nseg8.jsonl" --n_epochs 2

## Step 3: Query fine-tuned GPT3 Model for action sequence predictions

In [ ]:
#First replace the ground truth in validation files with recognition results.
path_to_recognition_file = ''
recognition_data = replace_to_recognition(path_to_recognition_file, val_data)
recognition_ds = build_finetune_dataset([recognition_data],save_jsonl=False)[0]

In [5]:
from openai_utils import query_ft_model
apikey = '' #you would need an open ai apikey
res = query_ft_model(apikey,recognition_ds)

## Post-process the responses and compute edit distance

In [7]:
import json
f = open('responses/cg/responses_recognition_nseg8_4.json')
res = json.load(f)

from post_processing import map_responses_to_index, clean_responses, get_label
bk_verb=dict({"put":1}) #for simplicity, we can use the most prevelent class as the bk_dict
bk_noun=dict({"dough":1})
verb_pred,noun_pred=map_responses_to_index(clean_responses(res),[verb2id,noun2id],[bk_verb,bk_noun])
verb_label,noun_label = get_label(val_data)

In [8]:
from edit_distance import edit_distance

print(f"Verb edit distance is: {edit_distance(verb_pred,verb_label)}")
print(f"Noun edit distance is: {edit_distance(noun_pred,noun_label)}")

Verb edit distance is: 0.7073052190818006
Noun edit distance is: 0.7185434425003738
